In [2]:
# %pwd

In [5]:
import os

In [4]:

os.chdir("../")

In [3]:
# %pwd

In [6]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
# Extract data from the PDF file
def load_pdf_file(path):
    loader= DirectoryLoader(path, 
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents= loader.load()
    return documents

In [10]:
extracted_data= load_pdf_file(path='Data/')

In [36]:
# extracted_data

In [39]:
# Split the data into chunks
def split_text(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [42]:
text_chunks= split_text(extracted_data)
print("Length of chunks: ", len(text_chunks))

Length of chunks:  5860


In [45]:
# text_chunks[0]

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings

In [16]:
# Download embeddings from HuggingFace
def download_embeddings():
    embeddings= HuggingFaceEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [17]:
embeddings= download_embeddings()

C:\Users\Faizan Habib\AppData\Local\Temp\ipykernel_9976\1643059553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings= HuggingFaceEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')


In [18]:
query_result= embeddings.embed_query("My world")
print("Length of query result: ", len(query_result))

Length of query result:  384


In [61]:
# query_result

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
PINECONE_API_KEY= os.getenv("PINECONE_API_KEY")
# print(PINECONE_API_KEY)

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medibot-index"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [16]:
# Embed each chunk and add to the Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [22]:
# Load existing index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch

In [60]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})

In [25]:
retrieved_docs = retriever.invoke("What is mole?")

In [ ]:
retrieved_docs

In [10]:
# Load Hugging Face API key
Llama_API_KEY = os.getenv("Llama_API_KEY")
# print(Llama_API_KEY)

In [82]:
from langchain_huggingface import HuggingFaceEndpoint


# Define Llama model
llm = HuggingFaceEndpoint(
    endpoint_url="https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-70B-Instruct",
    huggingfacehub_api_token=Llama_API_KEY,
    task="text-generation"  # Explicitly define the task
)

In [119]:
from langchain.prompts import PromptTemplate

# Define the system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved information to generate a response. "
    "If the retrieved information doesn't contain relevant content to answer the question,"
    "Say you don't have enough information to answer the question. "
    "Use five sentences maximum and keep the answer concise.\n\n{context}"
)

# Define the prompt template
prompt = PromptTemplate(
    input_variables=["context", "input"], 
    template="{{system_prompt}}\nUse the following context to answer the question:\n{context}\n\nQuestion: {input}\nAnswer:"
)


In [120]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create the combine docs chain
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# Create the retrieval chain properly
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [12]:
# Define the question
question = "What is vector?"

# Prepare the input for the chain and invoke
response = rag_chain.invoke({"input": question})

# Extract question and answer from the response
question = response.get('input', 'No question found')
answer = response.get('answer', 'No answer found')

# Print only the question and answer
print(f"Question: {question}")
print(f"Answer: {answer}")